In [1]:
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
import psycopg2 
import io

# Clean Trail Data

In [23]:
# Load trail info data
df_trail = pd.read_pickle('/Users/briangraham/insight/trailrec/data/trail_info_combined.pickle')

# Eliminate trails without descriptions
df_trail = df_trail[df_trail['description'].notnull()]

# subset trail data, eliminating uncommonly reported features with mostly NaN
df_trail = df_trail[['Altitude change', 'Altitude end', 'Altitude max', 'Altitude min',
       'Altitude start', 'Avg reverse time', 'Avg time', 'Bike type',
       'Difficulty rating', 'Direction', 'Distance climb', 'Distance down',
       'Global Ranking', 'Grade', 'Grade max', 'Grade min', 'Physical rating',
       'Ride in rain', 'Riding area', 'TTFs on trail', 'Trail Usage',
       'Trail type', 'avg_time', 'city', 'climb', 'descent', 'description',
       'distance', 'latitude', 'longitude', 'rating', 'state', 'votes',
       'Climb Difficulty', 'Distance flat']]
df_trail['votes'] = df_trail['votes'].str.replace(' votes','')
df_trail['votes'] = pd.to_numeric(df_trail['votes'])
df_trail['rating'] = df_trail['rating'].str.replace('Avg:','')
df_trail['rating'] = pd.to_numeric(df_trail['rating'])
df_trail['latitude'] = pd.to_numeric(df_trail['latitude'])
df_trail['longitude'] = pd.to_numeric(df_trail['longitude'])

## Have to go through this data and remove units from strings and convert to numbers

In [24]:
df_trail.head(n=1)

,Altitude change,Altitude end,Altitude max,Altitude min,Altitude start,Avg reverse time,Avg time,Bike type,Difficulty rating,Direction,...,descent,description,distance,latitude,longitude,rating,state,votes,Climb Difficulty,Distance flat
trail_id,,,,,,,,,,,,,,,,,,,,,
1-87-dh,"-1,508 ft",349 ft,"1,857 ft",349 ft,"1,857 ft",00:01:40,00:12:58,"DH, AM",Black Diamondrate,Downhill Primary,...,"-1,513 ft",Sustained steep and fast DH trail often used a...,2 miles,48.82592,-123.76254,4.55,BC,13,NaN,NaN


# Eliminate comments that do not match this trail subset

In [62]:
df_comments = pd.read_pickle('/Users/briangraham/insight/trailrec/data/trail_comments_combined.pickle')
df_comments_filtered = df_comments.join(df_trail,how='right')
df_comments_filtered = df_comments_filtered[['comment_votes','user','comment_text']]
df_comments_filtered = df_comments_filtered.dropna()

In [63]:
df_comments_filtered.head()

,comment_votes,user,comment_text
trail_id,,,
19th-hole,+ 1,blackcombterry,What's 19th Hole like below the Upper Legacy c...
2-3481,+ 0,atfarley,Just a road
2001,+ 1,amowat,Still some good ladders but alot gone. Big roc...
2001,+ 0,MtbSince84,Closed due to logging.
280-s,+ 0,HexAngel,Gently insistent ascent even this unfit geezer...


# Eliminate ridelogs that do not match this trail subset

In [66]:
df_ridelogs = pd.read_pickle('/Users/briangraham/insight/trailrec/data/trail_ridelogs_combined.pickle')
df_ridelogs_match = ridelogs.join(df_trail,how='right')
df_ridelogs_match = df_ridelogs_match[['ride_date','rider_url','rider_name','rider_state','rider_city','ride_difficulty_string']]
df_ridelogs_match.head()

,ride_date,rider_url,rider_name,rider_state,rider_city,ride_difficulty_string
trail_id,,,,,,
1-87-dh,Jul 5,https://www.trailforks.com/profile/BCpov/,BCpov,BC,Duncan,Very Difficult / Black Diamond
1-87-dh,Jul 5,https://www.trailforks.com/profile/canadaka/,canadaka,BC,Duncan,Very Difficult / Black Diamond
1-87-dh,May 27,https://www.trailforks.com/profile/campasternak/,campasternak,BC,Duncan,Very Difficult / Black Diamond
1-87-dh,May 21,https://www.trailforks.com/profile/campasternak/,campasternak,BC,Duncan,Very Difficult / Black Diamond
1-87-dh,May 20,https://www.trailforks.com/profile/superlightr...,superlightracer,BC,Duncan,Very Difficult / Black Diamond


In [67]:
df_ridelogs_match.shape

(1266757, 6)

# Save to Postgres

In [68]:
conn = psycopg2.connect("host=localhost dbname=postgres user=postgres")
cur = conn.cursor()